# Урок 2. Обзор основных алгоритмов машинного обучения, используемых в соревнованиях
Домашнее задание

В домашнем задании, будем обучать разные алгоритмы машинного обучения. Для этого, нужно предварительно зафиксировать схему валидации решения (можете выбрать любую, которую знаете). Выбранную схему валидации нужно использовать во всех задачах этого домашнего задания. Метрика качества - `ROC AUC`, целевая переменная - `isFraud`.

Ссылка на данные - https://drive.google.com/file/d/1gMEVl47pIoV1-AseB9doQ6DZNJrY3NkW/view?usp=sharing

##### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Задания основные

##### Загрузка данных

In [4]:
train = pd.read_csv('./data/assignment2_data/assignment_2_train.csv')
test = pd.read_csv('./data/assignment2_data/assignment_2_test.csv')
target = 'isFraud'

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))

train.shape = 180000 rows, 394 cols
test.shape = 100001 rows, 394 cols


##### Разделим  признаки `train` на числовые (непрерывные, дискртеные) и категориальные

In [5]:
all_features = train.drop(target, axis=1)
numerical_features = train.select_dtypes(include=[np.number]).drop(target, axis=1)
categorical_features = train.select_dtypes(include=[np.object])

print(f'count of all features in = {all_features.shape[1]}')
print(f"count of numeric features = {numerical_features.shape[1]}")
print(f"count of categorical features = {categorical_features.shape[1]}")

count of all features in = 393
count of numeric features = 379
count of categorical features = 14


##### Разделим numerical_features на x_train и x_valid

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(numerical_features,
                                                      train[target],
                                                      train_size=0.8,
                                                      random_state=42)

##### Сделаем get_dummies и добавим категориальные признаки к числовым

In [14]:
categorical_features_2 = categorical_features.copy()
categorical_features_2 = categorical_features_2.astype('category')

for i in ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']:
    categorical_features_2[i] = categorical_features_2[i].map({'T': 0, 'F': 1})

# categorical_features_2 = pd.get_dummies(categorical_features_2)
num_cat_features = pd.concat(
    [numerical_features,
     pd.get_dummies(categorical_features_2)], axis=1)

x_train_num_cat, x_valid_num_cat, y_train_num_cat, y_valid_num_cat = \
        train_test_split(num_cat_features, train[target], train_size=0.8, random_state=42)

### Задание 1: отобрать только числовые признаки и обучить модель XGBoost с параметром `booster=gbtree`. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [19]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": -1,
    "seed": 27
}

model_xgb_numeric = xgb.XGBClassifier(**params)
model_xgb_numeric.fit(X=x_train,
              y=y_train,
              eval_set=[(x_train, y_train), (x_valid, y_valid)],
              early_stopping_rounds=50,
              eval_metric="auc",
              verbose=20)

[0]	validation_0-auc:0.65232	validation_1-auc:0.63974
[20]	validation_0-auc:0.82254	validation_1-auc:0.81751
[40]	validation_0-auc:0.87410	validation_1-auc:0.86886
[60]	validation_0-auc:0.88985	validation_1-auc:0.88052
[80]	validation_0-auc:0.89506	validation_1-auc:0.88571
[100]	validation_0-auc:0.89986	validation_1-auc:0.89033
[120]	validation_0-auc:0.90421	validation_1-auc:0.89389
[140]	validation_0-auc:0.90699	validation_1-auc:0.89598
[160]	validation_0-auc:0.90934	validation_1-auc:0.89814
[180]	validation_0-auc:0.91030	validation_1-auc:0.89892
[200]	validation_0-auc:0.91030	validation_1-auc:0.89892
[219]	validation_0-auc:0.91030	validation_1-auc:0.89892


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=10, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=4,
              nthread=-1, num_parallel_tree=1, random_state=27, reg_alpha=0,
              reg_lambda=100, scale_pos_weight=1, seed=27, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Задание 2: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 1.

In [20]:
model_xgb_numeric_and_cat = xgb.XGBClassifier(**params)
model_xgb_numeric_and_cat.fit(X=x_train_num_cat,
                              y=y_train_num_cat,
                              eval_set=[(x_train_num_cat, y_train_num_cat),
                                        (x_valid_num_cat, y_valid_num_cat)],
                              early_stopping_rounds=50,
                              eval_metric="auc",
                              verbose=20)

[0]	validation_0-auc:0.65232	validation_1-auc:0.63974
[20]	validation_0-auc:0.82278	validation_1-auc:0.81779
[40]	validation_0-auc:0.87642	validation_1-auc:0.86819
[60]	validation_0-auc:0.89552	validation_1-auc:0.88377
[80]	validation_0-auc:0.90251	validation_1-auc:0.89051
[100]	validation_0-auc:0.90863	validation_1-auc:0.89588
[120]	validation_0-auc:0.91263	validation_1-auc:0.89965
[140]	validation_0-auc:0.91578	validation_1-auc:0.90269
[160]	validation_0-auc:0.91753	validation_1-auc:0.90389
[180]	validation_0-auc:0.91758	validation_1-auc:0.90400
[200]	validation_0-auc:0.91950	validation_1-auc:0.90564
[220]	validation_0-auc:0.91950	validation_1-auc:0.90564
[240]	validation_0-auc:0.91950	validation_1-auc:0.90564
[245]	validation_0-auc:0.91950	validation_1-auc:0.90564


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=10, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=4,
              nthread=-1, num_parallel_tree=1, random_state=27, reg_alpha=0,
              reg_lambda=100, scale_pos_weight=1, seed=27, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Задание 4: для числовых признаков обучить модель LightGBM. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [22]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 5000,
    #     "n_jobs": 6,
    "n_jobs": -1,
    "seed": 27
}

model_lgb_numeric = lgb.LGBMClassifier(**params)
model_lgb_numeric.fit(X=x_train,
                      y=y_train,
                      eval_set=[(x_train, y_train), (x_valid, y_valid)],
                      early_stopping_rounds=50,
                      eval_metric="auc",
                      verbose=50)

Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.877327	valid_1's auc: 0.86791
[100]	training's auc: 0.890733	valid_1's auc: 0.880059
[150]	training's auc: 0.902926	valid_1's auc: 0.890494


KeyboardInterrupt: 

### Задание 5: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 4.

In [ ]:
# Изменить x_train на x_train_cat

In [ ]:
# model_lgb_numeric_and_cat = lgb.LGBMClassifier(**params)
# model_lgb_numeric_and_cat.fit(X=x_train,
#                       y=y_train,
#                       eval_set=[(x_train, y_train), (x_valid, y_valid)],
#                       early_stopping_rounds=50,
#                       eval_metric="auc",
#                       verbose=10)

### Задание 6: обработать категориальные признаки встроенным методом в LightGBM. Выполнить задание 4. Сделать выводы о качестве работы алгоритма, по сравнению с пунктом 5.

In [87]:
x_train_lgb, x_valid_lgb, y_train_lgb, y_valid_lgb = train_test_split(pd.concat(
    [numerical_features,
     categorical_features_2], axis=1),
                                                      train[target],
                                                      train_size=0.8,
                                                      random_state=42)

In [ ]:
dtrain = lgb.Dataset(data=x_train, label=y_train)
dvalid = lgb.Dataset(data=x_valid, label=y_valid)

model_lgb_numeric_and_cat_auto = lgb.train(params=params,
                                           train_set=dtrain,
                                           num_boost_round=200,
                                           valid_sets=[dtrain, dvalid],
                                           categorical_feature=[
                                               'ProductCD', 'card4', 'card6',
                                               'P_emaildomain',
                                               'R_emaildomain', 'M1', 'M2',
                                               'M3', 'M4', 'M5', 'M6', 'M7',
                                               'M8', 'M9'
                                           ],
                                           early_stopping_rounds=50,
                                           verbose_eval=10)

### Задание 7: для числовых признаков обучить модель CatBoost. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

### Задание 8: обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 7.

### Задание 9: обработать категориальные признаки встроенным методом в CatBoost. Выполнить задание 7. Сделать выводы о качестве работы алгоритма, по сравнению с пунктом 8.

### Задание 10: построить ROC-кривую для всех построенных алгоритмов на обучающей и тестовой выборке. Сделать выводы о работе алгоритмов с точки зрения качества на тестовой выборке и с точки зрения переобучения.

In [ ]:
fig = plt.figure(figsize=(15, 5))
plt.plot(cv_result["train-auc-mean"], color="blue", linewidth=3, label="train")
plt.plot(cv_result["test-auc-mean"], color="orange", linewidth=3, label="test")
plt.fill_between(x=cv_result.index,
                 y1=cv_result["train-auc-mean"] - cv_result["train-auc-std"],
                 y2=cv_result["train-auc-mean"] + cv_result["train-auc-std"],
                 alpha=0.25,
                 color="blue")
plt.fill_between(x=cv_result.index,
                 y1=cv_result["test-auc-mean"] - cv_result["test-auc-std"],
                 y2=cv_result["test-auc-mean"] + cv_result["test-auc-std"],
                 alpha=0.25,
                 color="orange")
plt.title("XGBoost, CV-Results", size=14)
plt.xlabel("n_iterations", size=14)
plt.xlim(0, cv_result.index.max())
plt.ylabel("ROC-AUC", size=14)
plt.legend(loc="best")

## Задание на повторение:
Задание не обязательно к выполнению, но очень рекомендуется для понимания набора данных, этот набор данных будет использован и для следующего домашнего задания.

### Задание 1: Построить график распределения времени совершения транзакции для обучающей / тестовой выборки, сделать выводы о том, как разбиты данные и какие виды валидации могут подойти для данной задачи.

### Задание 2: построить scatter-plot зависимости суммы транзакции от времени совершения транзакции. Построить графики для обучающей выборки и для тестовой выборки, для обучающей выборки - построить как для целевой переменной = 0, так и для переменной = 1. Сделать выводы.

### Задание 3: построить распределение признака TransactionAmt в логарифмическом масштабе, сделать выводы о близости распредления к нормальному распределению. Построить распределение признака в логарифмическому масштабе для обучающей выборк и для тестовой выборки, сделать выводы.

### Задание 4: построить распределение признака целевой переменной в зависимости от значений категориальных признаков ProductCD, card4, card6. Сделать выводы.